In [26]:
import cv2
import numpy as np
import math

Reading input image

In [34]:
image = cv2.imread("input_image.png")

image_grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
image_padded = np.pad(image_grayscale, ((1, 1), (1, 1)), 'constant', constant_values=255)  # 1 pixel padding
blurred = cv2.GaussianBlur(image_padded, (5, 5), 0)  # 5x5 kernel gaussian blur
h, w = blurred.shape

Findings edges of the input image (Canny's algorithm)

In [ ]:
final = np.zeros((h, w), dtype=np.uint8)

# hyperparameter
low_thr = 6

# calculating gradient angle and intensivity for each pixel 

grad_angle = np.zeros((h, w))
grad_intense = np.zeros((h, w))
angles_list = [-180, -135, -90, -45, 0, 45, 90, 135, 180]

for i in range(1, h-1):
  for j in range(1, w-1):
    grad_x = (int(blurred[i][j + 1]) -  int(blurred[i][j - 1])) / 2
    grad_y = (int(blurred[i + 1][j]) -  int(blurred[i - 1][j])) / 2

    grad_intense[i][j] = math.sqrt(grad_x ** 2 + grad_y ** 2)
    grad_angle[i][j] = round(math.degrees(math.atan2(grad_y, grad_x)))


# deciding the direction of the gradient

for i in range(1, h-1):
    for j in range(1, w-1):

        pixel_angle = grad_angle[i][j]
        pixel_intense = grad_intense[i][j]
        angles_diff = [0 for _ in range(9)]

        for idx, angle in enumerate(angles_list):
            angles_diff[idx] = abs(pixel_angle - angle)

        angle = min(angles_diff)
        index =  angles_diff.index(angle)
        angle = angles_list[index]

        if angle == -180 or angle == 180 or angle == 0:
            pixel_1 = grad_intense[i][j - 1]
            pixel_2 = grad_intense[i][j + 1]
        elif angle == 45 or angle == -135:
            pixel_1 = grad_intense[i - 1][j - 1]
            pixel_2 = grad_intense[i + 1][j + 1]
        elif angle == 90 or angle == -90:
            pixel_1 = grad_intense[i - 1][j]
            pixel_2 = grad_intense[i + 1][j]
        elif angle == -45 or angle == 135:
            pixel_1 = grad_intense[i + 1][j - 1]
            pixel_2 = grad_intense[i - 1][j + 1]

        # checking if the pixel is considered as edge

        if pixel_intense > pixel_1 and pixel_intense > pixel_2 and pixel_intense > low_thr:
            final[i][j] = 255
        else:
            final[i][j] = 0

cv2.imwrite("edges.png", final)

True